<a href="https://colab.research.google.com/github/falls247/Colab/blob/main/Unknown_words_detection_01_tok_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google Driveのマウント
from google.colab import drive
# 強制リマウント
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
# ライブラリとモデルのインストール
!pip install spacy
!pip install transformers
!pip install ja_ginza
!pip install pytextspan

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.7/71.7 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 3.9 MB/s eta 0:00:00


In [3]:
# コーパスのパス（ドライブ上を想定）
corpus = "/content/drive/My Drive/Tokenizers/train_data_full.txt"
# 元になるトークナイザー保存先を指定
base_tokenizer_file = "/content/drive/My Drive/Tokenizers/base_tokenizer.json"
# トークナイザーの保存先を指定
tokenizer_file = "/content/drive/My Drive/Tokenizers/custum_tokenizer.json"

In [6]:
# SpaCyを使用したngramsのトークンを作成
"""形態素解析後にngrams表現で抽出する為のタスク"""
import spacy
from spacy.tokens import Doc

n = 1  # ngram数の設定
# モデルのロード。形態素解析に不要な機能は無効化しておく（高速化対応）
nlp = spacy.load("ja_ginza", disable=["ner", "tagger", "parser", "bunsetu_recognizer", "morphologizer", "compound_splitter", "tok2vec"])

# ngramの抽出（Token）
def get_ngrams_token(doc, n):
    ngrams = []
    for i in range(len(doc) - n + 1):
        ngram = doc[i:i + n]
        ngrams.append(ngram)
    return ngrams

# ngramの抽出（テキスト）
def get_ngrams_str(doc, n):
    ngrams = []
    for i in range(len(doc) - n + 1):
        ngram = doc[i:i + n]
        ngram_text = " ".join([token.text for token in ngram])  # ngramを文字列に変換
        ngram_text = ngram_text.replace(" ","") # 連結後の空白は削除しておく
        ngrams.append(ngram_text)
    return ngrams

# spacyのカスタムコンポーネントにngram抽出を追加
Doc.set_extension("ngrams_token", getter=lambda doc: get_ngrams_token(doc, n), force=True)
Doc.set_extension("ngrams_str", getter=lambda doc: get_ngrams_str(doc, n), force=True)

# spacyを使った形態素解析（戻り値はspanオブジェクト）※検証用
def spacy_token(sentence):
    doc = nlp(sentence)
    return doc._.ngrams_token

# spacyを使った形態素解析（戻り値はstr型）
def spacy_str(sentence):
    doc = nlp(sentence)
    return doc._.ngrams_str

# n=2以上の時、元文章の表示がおかしくならないようspacyでトークン化後に最初のトークンのみを返す為の関数
def spacy_nlp(sentence):
    doc = nlp(sentence)
    words = [word.text for word in doc]
    return words[0]

# 結果と変数の型を確認
test = "この文章の場合はどうなるかな。"
print(f"ngramのn数が{n}の時は以下の結果になります。")
print(test)
print(spacy_token(test))
print(f"spacy_tokenでの型は：{type(spacy_token(test)[0])}")
print(spacy_str(test))
print(f"spacy_strでの型は：{type(spacy_str(test))}")
print(spacy_nlp(test))
print(f"spacy_nlpでの型は：{type(spacy_nlp(test))}")

ngramのn数が1の時は以下の結果になります。
この文章の場合はどうなるかな。
[この, 文章, の, 場合, は, どう, なる, か, な, 。]
spacy_tokenでの型は：<class 'spacy.tokens.span.Span'>
['この', '文章', 'の', '場合', 'は', 'どう', 'なる', 'か', 'な', '。']
spacy_strでの型は：<class 'list'>
この
spacy_nlpでの型は：<class 'str'>


In [17]:
# コーパスの前処理
"""初回のみ実行。語彙の追加は前処理済みのデータに追加していく。"""

from tqdm.auto import tqdm
import pandas as pd

# 空白行を削除しながらファイルを読み込む
lines = [line.strip() for line in open(corpus, "r", encoding="UTF-8") if line.strip()]

# 語彙を格納するリストを作成
vocab_list = []

# 語彙を分割してリストに追加
for line in tqdm(lines):
    vocab_list.extend(spacy_str(line))

# 重複を削除
vocab_set = set(vocab_list)

# データフレームに変換
df = pd.DataFrame(vocab_set, columns=["vocab"])

  0%|          | 0/19568 [00:00<?, ?it/s]

In [19]:
# ベースとなるトークナイザ用jsonファイルの中身
base_tok = """{
  "version": "1.0",
  "truncation": null,
  "padding": null,
  "added_tokens": [
    {
      "id": 0,
      "content": "[PAD]",
      "single_word": false,
      "lstrip": false,
      "rstrip": false,
      "normalized": false,
      "special": true
    },
    {
      "id": 1,
      "content": "[UNK]",
      "single_word": false,
      "lstrip": false,
      "rstrip": false,
      "normalized": false,
      "special": true
    },
    {
      "id": 2,
      "content": "[CLS]",
      "single_word": false,
      "lstrip": false,
      "rstrip": false,
      "normalized": false,
      "special": true
    },
    {
      "id": 3,
      "content": "[SEP]",
      "single_word": false,
      "lstrip": false,
      "rstrip": false,
      "normalized": false,
      "special": true
    },
    {
      "id": 4,
      "content": "[MASK]",
      "single_word": false,
      "lstrip": false,
      "rstrip": false,
      "normalized": false,
      "special": true
    }
  ],
  "normalizer": {
    "type": "BertNormalizer",
    "clean_text": true,
    "handle_chinese_chars": false,
    "strip_accents": false,
    "lowercase": true
  },
  "pre_tokenizer": {
    "type": "BertPreTokenizer"
  },
  "post_processor": null,
  "decoder": {
    "type": "WordPiece",
    "prefix": "##",
    "cleanup": true
  },
  "model": {
    "type": "WordPiece",
    "unk_token": "[UNK]",
    "continuing_subword_prefix": "##",
    "max_input_chars_per_word": 100,
    "vocab": {
      "[PAD]": 0,
      "[UNK]": 1,
      "[CLS]": 2,
      "[SEP]": 3,
      "[MASK]": 4
    }
  }
}"""

# ベースとなるトークナイザ用jsonファイルを作成
with open(base_tokenizer_file, "w") as f:
    f.write(base_tok)


In [20]:
# ベースとなるjsonファイルを読み込む
"""Google Drive上で作業する時はファイルの反映まで時間がかかる場合があるので注意"""
import json
with open(base_tokenizer_file, "r") as f:
  data = json.load(f)

# dfのvocab列に入っている単語を取得する
words = df["vocab"].tolist()

# jsonファイルのvocabに単語を追記する
vocab = data["model"]["vocab"]
index = len(vocab) # 現在の語彙数
for word in words:
  vocab[word] = index # 単語に新しいインデックスを割り当てる
  index += 1 # インデックスを更新する

# トークン化用の新しいjsonファイルとして保存する
with open(tokenizer_file, "w") as f:
  json.dump(data, f, ensure_ascii=False, indent=2)


In [21]:
# カスタムトークナイザーを使用する為の準備
import textspan
from typing import List, Optional
from tokenizers import NormalizedString, PreTokenizedString, Tokenizer
from tokenizers.pre_tokenizers import BertPreTokenizer, PreTokenizer

# 検出用のトークナイザー
class WorkPreTokenizer:

    def tokenize(self, sequence: str) -> List[str]:
      return spacy_str(sequence)

    def custom_split(
        self, i: int, normalized_string: NormalizedString
    ) -> List[NormalizedString]:
        """See. https://github.com/huggingface/tokenizers/blob/b24a2fc/bindings/python/examples/custom_components.py"""
        text = str(normalized_string)
        tokens = self.tokenize(text)
        tokens_spans = textspan.get_original_spans(tokens, text)
        return [
            normalized_string[st:ed]
            for char_spans in tokens_spans
            for st, ed in char_spans
        ]

    def pre_tokenize(self, pretok: PreTokenizedString):
        pretok.split(self.custom_split)


def load_custom_tokenizer(tokenizer_file: str) -> Tokenizer:
    """Tokenizerのロード処理：tokenizer.json からTokenizerをロードし、custome PreTokenizerをセットする。"""
    tok = Tokenizer.from_file(tokenizer_file)
    # ダミー注入したRustベースのPreTokenizerを、custom PreTokenizerで上書き。
    tok.pre_tokenizer = PreTokenizer.custom(WorkPreTokenizer())
    return tok


In [22]:
# 検証テキストの格納
text = """

誤変換や誤字を見つけてくれるかのテスト。
ウザギはウサギの間違いです。

"""

In [26]:
# 誤字、誤変換の検出タスク
from tqdm.auto import tqdm
import re

# tokオブジェクトの生成
tok = load_custom_tokenizer(tokenizer_file)
tok = Tokenizer.from_file(tokenizer_file)
tok.pre_tokenizer = PreTokenizer.custom(WorkPreTokenizer())

# エスケープシーケンスの辞書を作る
color_dic = {'red_bg':'\033[41m', 'white_text':'\033[37m', 'reset_bg':'\033[49m', 'reset_text':'\033[0m'}

# 変数初期化
prep = [] # 検証テキスト格納用
temp = [] # トークナイズ後の語彙確認用
word_pos = [] # [UNK]がHITした位置格納
unk_list = [] # [UNK]がHITした文字列格納
row_no = 0 # 行番号
r = 0 # リスト内文字列特定用

# 検証テキストをリストに格納
for line in text.split("\n"):
  # スペースはトークン化できないので削除しながら格納していく
  prep.append(re.sub(r"\s+", "", line))

# リストから一行ずつトークナイザーにかけてtempに格納
for line in tqdm(prep):
  line = re.sub(r'^\s*', '', line)
  temp = tok.encode(line).tokens
  if temp:
    i = 0

    # [UNK]があったら位置を格納
    for word in temp:
      if word == "[UNK]":
        word_pos.append(i)
      i += 1

    if word_pos:
      # リストが空でなければ形態素解析した結果をtempに格納
      temp = spacy_str(line)
      for r in range(len(temp)-1):
        if r in word_pos:
          unk_list.append(temp[r]) # UNK判定のリストに追加
          temp[r] = color_dic['red_bg'] + color_dic['white_text'] + spacy_nlp(temp[r]) + color_dic['reset_bg'] + color_dic['reset_text']
        else:
          temp[r] = spacy_nlp(temp[r])
      line = ''.join(temp).replace("\n","")
      word_pos = []

    # 結果の出力
    print(f"{row_no:06d}:{line}")

    row_no += 1 # 行番号加算


# UNKになった語彙のリストを集合に変換して重複を除去する
unk_list = list(set(unk_list))

print(f"検出数　　：{len(unk_list)}")
print(f"検出文字列：{unk_list}")

  0%|          | 0/6 [00:00<?, ?it/s]

000000:誤変換や誤字を見つけてくれるかのテスト。
000001:ウザギはウサギの間違いです。
検出数　　：1
検出文字列：['ウザギ']


In [24]:
# コーパスによって最初は誤検出が多くなる為、未知語リストにある言葉をトークナイザ用JSONファイルに追記する
"""正しく未知語と検出されたものまでJSONファイルに反映すべきではないので、対象外とする（正しく検出できた）文字列を入力する"""

# 未知語リストをグローバル変数として定義
global unk_list

# 誤検出ではなかった文字を入力（何もない場合はそのままENTER）
print("検出して正解の文字列（誤字、誤変換）")
target = input()

# 未知語リストから削除
if target in unk_list:
    unk_list.remove(target)

# 結果を表示
print(f"正しい語彙として登録する数　　：{len(unk_list)}")
print(f"正しい語彙として登録する文字列：{unk_list}")


検出して正解の文字列（誤字、誤変換）
ウザギ
正しい語彙として登録する数　　：3
正しい語彙として登録する文字列：['誤変換', 'ウサギ', '誤字']


In [25]:
# 未知語リストをトークナイズに使用する語彙として登録するタスク

# ベースとなるjsonファイルを読み込む
import json
with open(tokenizer_file, "r") as f:
  data = json.load(f)

# jsonファイルのvocabに単語を追記する
vocab = data["model"]["vocab"]
index = len(vocab) # 現在の語彙数
for word in unk_list:
  vocab[word] = index # 単語に新しいインデックスを割り当てる
  index += 1 # インデックスを更新する

# トークン化用の新しいjsonファイルとして保存する
with open(tokenizer_file, "w") as f:
  json.dump(data, f, ensure_ascii=False, indent=2)